<a href="https://colab.research.google.com/github/4lcatrust/portfolio/blob/main/OpenPowerlifting_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import polars as pl
import os
from google.oauth2 import service_account
from googleapiclient.discovery import build
import kagglehub

In [2]:
# Download Kaggle OpenPowerlifting dataset and read with Polars
path = kagglehub.dataset_download("open-powerlifting/powerlifting-database")
filename = 'openpowerlifting-2024-01-06-4c732975.csv'
filepath = f'{path}/{filename}'

100%|██████████| 176M/176M [00:03<00:00, 49.6MB/s]

Extracting files...


In [3]:
# Read dataset
df = pl.read_csv(path+'/openpowerlifting-2024-01-06-4c732975.csv', infer_schema_length = 1000)

# Check shape of dataset
print(f'Dataset Row Count: {df.shape[0]:,}')
print(f'Dataset Column Count: {df.shape[1]:,}')

# Preview dataset
display(df.head())

# Checking on dataset's schema to enhance data type
display(dict(df.schema))

# Checking on dataset's null values on main column, in this case, name of the athletes
print('Is there any NULL values in `Name` column?: ' + str(df['Name'].is_empty()))

# Casting `Age` from float into int data type
df = df.with_columns(pl.col('Age').cast(pl.Int64, strict=False))

# Checking if the transformed schema are succesfully transformed or not
display(dict(df[['Age']].schema))

Dataset Row Count: 3,043,013
Dataset Column Count: 41


Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,Bench4Kg,Best3BenchKg,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Place,Dots,Wilks,Glossbrenner,Goodlift,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
str,str,str,str,f64,str,str,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,str,str,str,str,str,str,str,str,str
"""Alona Vladi""","""F""","""SBD""","""Raw""",33.0,"""24-34""","""24-39""","""O""",58.3,"""60""",75.0,80.0,-90.0,null,80.0,50.0,55.0,60.0,null,60.0,95.0,105.0,107.5,null,107.5,247.5,"""1""",279.44,282.18,249.42,57.1,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournament"""
"""Galina Solovyanova""","""F""","""SBD""","""Raw""",43.0,"""40-44""","""40-49""","""M1""",73.1,"""75""",95.0,100.0,105.0,null,105.0,62.5,67.5,-72.5,null,67.5,100.0,110.0,-120.0,null,110.0,282.5,"""1""",278.95,272.99,240.35,56.76,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournament"""
"""Daniil Voronin""","""M""","""SBD""","""Raw""",15.5,"""16-17""","""14-18""","""T""",67.4,"""75""",85.0,90.0,100.0,null,100.0,55.0,62.5,-65.0,null,62.5,90.0,100.0,105.0,null,105.0,267.5,"""1""",206.4,206.49,200.45,41.24,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournament"""
"""Aleksey Krasov""","""M""","""SBD""","""Raw""",35.0,"""35-39""","""24-39""","""O""",66.65,"""75""",125.0,132.0,137.5,null,137.5,115.0,122.5,-127.5,null,122.5,150.0,165.0,170.0,null,170.0,430.0,"""1""",334.49,334.94,325.32,66.68,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournament"""
"""Margarita Pleschenkova""","""M""","""SBD""","""Raw""",26.5,"""24-34""","""24-39""","""O""",72.45,"""75""",80.0,85.0,90.0,null,90.0,40.0,50.0,-60.0,null,50.0,112.5,120.0,125.0,null,125.0,265.0,"""1""",194.46,193.55,187.29,39.34,"""Yes""","""Russia""",null,"""GFP""",null,"""2019-05-11""","""Russia""",null,"""Bryansk""","""Open Tournament"""


{'Name': String,
 'Sex': String,
 'Event': String,
 'Equipment': String,
 'Age': Float64,
 'AgeClass': String,
 'BirthYearClass': String,
 'Division': String,
 'BodyweightKg': Float64,
 'WeightClassKg': String,
 'Squat1Kg': Float64,
 'Squat2Kg': Float64,
 'Squat3Kg': Float64,
 'Squat4Kg': Float64,
 'Best3SquatKg': Float64,
 'Bench1Kg': Float64,
 'Bench2Kg': Float64,
 'Bench3Kg': Float64,
 'Bench4Kg': Float64,
 'Best3BenchKg': Float64,
 'Deadlift1Kg': Float64,
 'Deadlift2Kg': Float64,
 'Deadlift3Kg': Float64,
 'Deadlift4Kg': Float64,
 'Best3DeadliftKg': Float64,
 'TotalKg': Float64,
 'Place': String,
 'Dots': Float64,
 'Wilks': Float64,
 'Glossbrenner': Float64,
 'Goodlift': Float64,
 'Tested': String,
 'Country': String,
 'State': String,
 'Federation': String,
 'ParentFederation': String,
 'Date': String,
 'MeetCountry': String,
 'MeetState': String,
 'MeetTown': String,
 'MeetName': String}

Is there any NULL values in `Name` column?: False


{'Age': Int64}

In [4]:
# Create ingestion into spreadsheet
from google.colab import userdata

# Use Google service account credentials for with enabled Spreadsheet API
SERVICE_ACCOUNT = json.loads(userdata.get('4LCA_SS_API_KEY'))
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
creds = service_account.Credentials.from_service_account_info(SERVICE_ACCOUNT, scopes=SCOPES)

# Input ID and range of a targeted spreadsheet
SPREADSHEET_ID = userdata.get('SPREADSHEET_ID')
SPREADSHEET_RANGE = 'OpenPowerlifting'
service = build('sheets', 'v4', credentials=creds)

In [6]:
# Insert spreadsheet count cells limitation
spreadsheet_limit_cells = round(10000000 * 0.75)
dataset_maximum_row = round(spreadsheet_limit_cells / df.shape[1])

# Create batching process
batch_size = round(dataset_maximum_row / 5)
iter = 0
while iter <= dataset_maximum_row:
    print(f'Iteration {iter} starting...')
    match iter:
        case 0:
            # For first batch, insert column name for header
            data = df[iter : iter + batch_size].rows()
            data.insert(0, df.columns)
            body = {
                'values': data
            }
            result = service.spreadsheets().values().update(
                                                            spreadsheetId = SPREADSHEET_ID,
                                                            range = SPREADSHEET_RANGE,
                                                            valueInputOption = 'RAW',
                                                            body = body
                                                            ).execute()
            print(f"Header appended. {result['updatedCells']} cells updated.")
        case _:
            # For the rest of the batch only append the spreadsheet
            data = df[iter : iter + batch_size].rows()
            body = {
                'values': data
            }

            result = service.spreadsheets().values().append(
                                                            spreadsheetId = SPREADSHEET_ID,
                                                            range = SPREADSHEET_RANGE,
                                                            valueInputOption = 'RAW',
                                                            insertDataOption = 'INSERT_ROWS',
                                                            body = body
                                                            ).execute()
            print(f"Batch appended. {result['updates']['updatedCells']} cells updated.")
    iter += batch_size

print(f'Data ingestion done\n')

# Check data ingestion result
request = service.spreadsheets().get(spreadsheetId = SPREADSHEET_ID, fields = 'sheets(properties(gridProperties(rowCount,columnCount)))')
response = request.execute()
for sheet in response['sheets']:
    row_count = sheet['properties']['gridProperties']['rowCount']
    column_count = sheet['properties']['gridProperties']['columnCount']
    print(f'Row Count: {row_count} ingested')
    print(f'Column Count: {column_count} ingested')

Iteration 0 starting...
Header appended. 1009115 cells updated.
Data ingestion done
Iteration 36585 starting...
Batch appended. 1051031 cells updated.
Data ingestion done
Iteration 73170 starting...
Batch appended. 1079798 cells updated.
Data ingestion done
Iteration 109755 starting...
Batch appended. 1111956 cells updated.
Data ingestion done
Iteration 146340 starting...
Batch appended. 1069523 cells updated.
Data ingestion done
Iteration 182925 starting...
Batch appended. 1065259 cells updated.
Data ingestion done
Row Count: 219511 ingested
Column Count: 41 ingested
